In [1]:
import os
import random
from time import time
import pandas as pd
import numpy as np
import networkx as nx
import graph_tool.all as gt
from motif_counts import *
from tqdm.auto import tqdm
from collections import defaultdict
from tqdm import tqdm
import pickle

/lmb/home/fkampf/miniconda3/envs/py312_jefferis/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy

In [3]:
df = pd.read_feather('mcns_fw_edge_comp.feather')

In [4]:
# 1) build a unique list of all node labels
labels = np.unique(np.concatenate([df['pre'].values, df['post'].values]))

# 2) create the graph and a string vertex‐property to store the label
g = gt.Graph(directed=True)
v_label = g.new_vp("string")
g.vp["label"] = v_label

# 3) add one vertex per label, keep a Python dict to map label→vertex
label2v = {}
for L in labels:
    v = g.add_vertex()
    label2v[L] = v
    v_label[v] = str(L)

# 4) create a float edge‐property for your weights
e_weight = g.new_ep("float")
g.ep["weight"] = e_weight

# 5) add all edges with their weights
edge_list = [
    (label2v[src], label2v[tgt], float(w))
    for src, tgt, w in df[['pre','post','weight_m']].itertuples(index=False)
]
g.add_edge_list(edge_list, eprops=[g.ep["weight"]])
loops = [e for e in g.edges() if e.source() == e.target()]
for e in loops:
    g.remove_edge(e)

In [5]:
V = list(range(g.num_vertices()))
E = {
    (int(e.source()), int(e.target()))
    for e in g.edges()
    if g.ep['weight'][e] >= 5
}

In [ ]:
import graph_tool.all as gt
from graph_tool.clustering import motifs, motif_significance
from graph_tool.all import GraphView
from graph_tool.all import Graph
from graph_tool.clustering import motifs    
from graph_tool.draw import graph_draw



keep = g.new_vertex_property("bool")
keep.a[:] = False
keep.a[:500] = True

# view that hides all other vertices
view = GraphView(g, vfilt=keep)

# copy into a standalone graph, dropping hidden bits
g100 = Graph(view, prune=True)



motifs_list, counts, maps = motifs(g, 3, return_maps=True)



for i, (m, c) in enumerate(zip(motifs_list, counts)):
    # extract the edge list of this motif
    edges = [(int(e.source()), int(e.target())) for e in m.edges()]
    print(f"Motif {i}: edges = {edges}, count = {c}")

    # if you’d like to visualize it:
    graph_draw(
        m,
        vertex_fill_color=[0.5,0.5,0.5,1], 
        output_size=(200,200),
        output=f"{i}.png"  # omit or set a filename like "motif_{i}.png"
    )

with open('motifs_list.pkl', 'wb') as f:
    # protocol=pickle.HIGHEST_PROTOCOL uses the most efficient format
    pickle.dump(motifs_list, f, protocol=pickle.HIGHEST_PROTOCOL)
    
with open('counts.pkl', 'wb') as f:
    # protocol=pickle.HIGHEST_PROTOCOL uses the most efficient format
    pickle.dump(counts, f, protocol=pickle.HIGHEST_PROTOCOL)

with open('maps.pkl', 'wb') as f:
    # protocol=pickle.HIGHEST_PROTOCOL uses the most efficient format
    pickle.dump(maps, f, protocol=pickle.HIGHEST_PROTOCOL)

